# 03 - Interactive Viz

## Deadline

Wednesday November 8th, 2017 at 11:59PM

## Important Notes

- Make sure you push on GitHub your Notebook with all the cells already evaluated
- Note that maps do not render in a standard Github environment : you should export them to HTML and link them in your notebook.
- Remember that `.csv` is not the only data format. Though they might require additional processing, some formats provide better encoding support.
- Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
- Please write all your comments in English, and use meaningful variable names in your code

## Background

In this homework we will be exploring interactive visualization, which is a key ingredient of many successful data visualizations (especially when it comes to infographics).

Unemployment rates are major economic metrics and a matter of concern for governments around the world. Though its definition may seem straightforward at first glance (usually defined as the number of unemployed people divided by the active population), it can be tricky to define consistently. For example, one must define what exactly unemployed means : looking for a job ? Having declared their unemployment ? Currently without a job ? Should students or recent graduates be included ? We could also wonder what the active population is : everyone in an age category (e.g. `16-64`) ? Anyone interested by finding a job ? Though these questions may seem subtle, they can have a large impact on the interpretation of the results : `3%` unemployment doesn't mean much if we don't know who is included in this percentage. 

In this homework you will be dealing with two different datasets from the statistics offices of the European commission ([eurostat](http://ec.europa.eu/eurostat/data/database)) and the Swiss Confederation ([amstat](https://www.amstat.ch)). They provide a variety of datasets with plenty of information on many different statistics and demographics at their respective scales. Unfortunately, as is often the case is data analysis, these websites are not always straightforward to navigate. They may include a lot of obscure categories, not always be translated into your native language, have strange link structures, â€¦ Navigating this complexity is part of a data scientists' job : you will have to use a few tricks to get the right data for this homework.

For the visualization part, install [Folium](https://github.com/python-visualization/folium) (*HINT*: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!). Folium's `README` comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find two `.topojson` files, containing the geo-coordinates of 

- European countries (*liberal definition of EU*) (`topojson/europe.topojson.json`, [source](https://github.com/leakyMirror/map-of-europe))
- Swiss cantons (`topojson/ch-cantons.topojson.json`) 

These will be used as an overlay on the Folium maps.

## Assignment

1. Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

2. Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

3. Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

4. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [RÃ¶stigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?

In [ ]:
import os
import pandas as pd
import json
import folium
import branca
import numpy as np
print(folium.__version__ == '0.5.0')


In [ ]:
DATA_FOLDER = './data'
EU_DATA_FOLDER=DATA_FOLDER + '/LFS-quaterly-TotalUnemployment'

# LFS: Labour force survey
#URGAN: unemployement rates by gender, age and nationality
#separated sheet, use sheet= Data with the total (male + female)
urgan_data = pd.read_excel(EU_DATA_FOLDER + '/lfsq_urgan-2.xls', sheetname='Data', header=11);

#keep most recent unemployement rates i.e. from second quarter of 2017
urgan = urgan_data[['GEO/TIME','2017Q2']]

#drop  Europe stats + ":"+"nan" + Sepcial values
urgan= urgan.drop([0,1,2,3,4,5,41,39,40],axis=0)
#change columns name
urgan= urgan.rename(columns={'GEO/TIME': 'Country', '2017Q2': 'Unemployement (%)'})

#change countries name to merge with the topojson data later
urgan = urgan.set_value(10,'Country','Germany')
urgan = urgan.set_value(37,'Country','The former Yugoslav Republic of Macedonia')

#create data frame with countries id  and names from the topo
topo= pd.read_json('./topojson/europe.topojson.json',typ= 'series')
eu_topo= pd.DataFrame(topo['objects']['europe']['geometries'])
#get ids and names
countries_id=eu_topo['id']
countries_name=pd.DataFrame(list(eu_topo['properties'].values))
#create df with id and name
eu_code=pd.DataFrame()
eu_code['id']=countries_id
eu_code['Country']=countries_name['NAME']
eu_code =eu_code.sort_values('Country')
eu_code.head(100)

#merge ids and unemployement rates using the country's name
urgan = pd.merge(eu_code,urgan_recent, 'inner')
urgan = urgan.set_index('id')
urgan

In [ ]:
m_eu = folium.Map([46, 6], tiles='cartodbpositron', zoom_start=4)

unemployed_series = urgan['Unemployement (%)']
colorscale = branca.colormap.linear.YlGnBu.scale(0, 30)

def style_function(feature):
    unemployed = unemployed_series.get(feature['id'][-5:], None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if unemployed is None else colorscale(unemployed)
    }

folium.TopoJson(open('./topojson/europe.topojson.json'),
                'objects.europe',style_function=style_function).add_to(m_eu)

m_eu